In [2]:
#Importing libraries 
import numpy as np
#import pandas as pd
import nibabel as nib
import scipy as sp
import matplotlib.pyplot as plt
#import seaborn as sb
import nipy, os
%matplotlib inline 
os.chdir ("/Users/arashnazeri/Desktop/mri-python/")

ValueError: numpy.dtype has the wrong size, try recompiling

In [ ]:
#Loading Data & Reshaping
data=nib.load("func.nii.gz")
dims=data.shape
affine=data.get_affine()
data=data.get_data()
data=data.reshape(dims[0]*dims[1]*dims[2],dims[3])
rois=nib.load("rois.nii.gz").get_data()
rois=rois.reshape(dims[0]*dims[1]*dims[2],1)

In [ ]:
#Extracting time series
idx = np.where(rois==1)[0]
x = data[idx,:]
ts = np.mean(x,axis=0)
plt.plot(ts)

In [ ]:
#Seed-based Correlation
output=np.zeros(dims[0]*dims[1]*dims[2])
for i in range(dims[0]*dims[1]*dims[2]):
    output[i]=np.corrcoef(ts,data[i,:])[1,0]
    
output[np.isnan(output)]=0

In [ ]:
#Reshape
final=output.reshape(dims[0],dims[1],dims[2])
mask=nib.load("mask.nii.gz").get_data()
masked=final*mask
ni_out=nib.nifti1.Nifti1Image(masked,affine)
ni_out.to_filename("seed_corr.nii.gz")

In [ ]:
#Printing Output
plt.imshow(final[:,:,40]*mask[:,:,40],cmap=plt.cm.RdBu_r)


In [ ]:
out_matrix=np.zeros((len(np.unique(rois))-1,dims[3]))

for roi in np.unique(rois):
    if roi > 0:
        idx = np.where(rois==roi)[0]
        x = data[idx,:]
        ts = np.mean(x,axis=0)
        ts=ts/np.std(ts);ts=ts-np.mean(ts)        
        out_matrix[roi-1,:]=ts


In [ ]:
plt.plot(out_matrix.T,)
plt.show

In [ ]:
G=np.corrcoef(out_matrix)
plt.imshow(G,interpolation='nearest',cmap=plt.cm.RdBu_r,vmin=-1,vmax=1)
plt.colorbar()

In [ ]:
%reset